Для реализации данных требований добавим несколько интерактивных элементов в наш Streamlit-дэшборд. Эти элементы позволят пользователю выбирать станции и временные интервалы, а также будут визуализировать загруженность станций и другие ключевые метрики.

Обновленный код Streamlit-дэшборда

In [ ]:
import streamlit as st
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
from datetime import datetime

# Настройка подключения к базе данных PostgreSQL
DB_HOST = 'your_db_host'
DB_NAME = 'your_db_name'
DB_USER = 'your_db_user'
DB_PASS = 'your_db_password'

# Создание подключения к базе данных
@st.cache(allow_output_mutation=True)
def get_db_connection():
    engine = create_engine(f'postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}/{DB_NAME}')
    return engine

# Загрузка данных из базы данных
@st.cache
def load_data():
    engine = get_db_connection()
    query = "SELECT * FROM your_table_name"  # Измените на ваше название таблицы
    df = pd.read_sql(query, engine)
    return df

# Фильтрация данных по выбранным станциям и временным интервалам
def filter_data(df, stations, start_time, end_time):
    filtered_df = df[
        (df['станция входа'].isin(stations) | df['станция выхода'].isin(stations)) &
        (df['время входа'] >= start_time) &
        (df['время выхода'] <= end_time)
    ]
    return filtered_df

# Основная функция для создания дашборда
def main():
    st.title("Аналитический дашборд")

    # Загрузка данных
    df = load_data()
    df['время входа'] = pd.to_datetime(df['время входа'])
    df['время выхода'] = pd.to_datetime(df['время выхода'])
    df['длительность поездки'] = df['время выхода'] - df['время входа']

    # Интерфейс для выбора станций и временного интервала
    stations = st.multiselect('Выберите станции', df['станция входа'].unique())
    start_time = st.slider('Начальное время', min_value=datetime(2023, 6, 1, 0, 0), max_value=datetime(2023, 6, 1, 23, 59), value=datetime(2023, 6, 1, 8, 0))
    end_time = st.slider('Конечное время', min_value=datetime(2023, 6, 1, 0, 0), max_value=datetime(2023, 6, 1, 23, 59), value=datetime(2023, 6, 1, 20, 0))

    # Фильтрация данных
    filtered_df = filter_data(df, stations, start_time, end_time)

    # Вычисление метрик для выбранного интервала
    station_load = filtered_df['станция входа'].value_counts() + filtered_df['станция выхода'].value_counts()
    max_load = station_load.max()
    station_load_percent = (station_load / max_load) * 100 if max_load != 0 else 0
    average_capacity = station_load.mean()
    top_stations = station_load.nlargest(5)
    entry_count = filtered_df['станция входа'].value_counts()
    exit_count = filtered_df['станция выхода'].value_counts()
    average_trip_duration = filtered_df['длительность поездки'].mean()

    # Визуализация данных
    st.header("Основные метрики")
    st.metric("Средняя пропускная способность", f"{average_capacity:.2f}")
    st.metric("Средняя продолжительность поездки", f"{average_trip_duration}")

    st.header("Загруженность станций (%)")
    st.bar_chart(station_load_percent)

    st.header("Топ загруженных станций")
    st.bar_chart(top_stations)

    st.header("Количество входов на станции")
    st.bar_chart(entry_count)

    st.header("Количество выходов на станции")
    st.bar_chart(exit_count)

    st.header("Данные по поездкам")
    st.dataframe(filtered_df)

if __name__ == "__main__":
    main()


Дополнения:
Интерфейс для выбора станций и временного интервала:

st.multiselect позволяет выбрать одну или несколько станций.
st.slider позволяет выбрать начальное и конечное время.
Фильтрация данных:

filter_data фильтрует данные по выбранным станциям и временным интервалам.
Вычисление метрик:

Загруженность станций в процентах относительно максимальной загруженности.
Топ самых загруженных станций.
Средняя пропускная способность.
Средняя продолжительность поездки.
Визуализация данных:

Графики и метрики, отображающие ключевые показатели

Этот обновленный дашборд позволяет пользователю проводить анализ загруженности станций в течение дня, задавать временные интервалы и выбирать станции, а также визуализировать ключевые метрики и динамику этих метрик во времени.